In [1]:
import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

### data loaders

In [2]:
class csv_data(Dataset):
    
    def __init__(self, filename):
        self.dataframe = pd.read_csv(filename).astype('float32')
        return None
    
    def __getitem__(self, index):
        X, y = self.dataframe.iloc[index,:-1], self.dataframe.iloc[index,-1]
        y = np.atleast_1d(np.array(y, dtype=np.int64))
        X, y = torch.from_numpy(np.array(X, dtype=np.float64)), torch.LongTensor(y).squeeze()
        return X.float(), y.squeeze()
    
    def __len__(self):
        return self.dataframe.shape[0]

In [3]:
train_resnet = csv_data('../../data/plant-seed/data/features/train_resnet_50.csv')
valid_resnet = csv_data('../../data/plant-seed/data/features/valid_resnet_50.csv')

In [4]:
train_resnet = DataLoader(train_resnet, batch_size=12, shuffle=True, num_workers=1)
valid_resnet = DataLoader(valid_resnet, batch_size=64, shuffle=False, num_workers=1)

### model

In [11]:
class resnet(torch.nn.Module):

    def __init__(self):
        super(resnet, self).__init__()
        self.dense1 = torch.nn.Linear(2048, 500)
        self.dropout = torch.nn.Dropout(p=0.33)
        self.batchnorm = torch.nn.BatchNorm1d(num_features=500)
        self.dense2 = torch.nn.Linear(500, 12)
        return None

    def forward(self, x):
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.batchnorm(x)
        x = self.dense2(x)
        return x
    
model = resnet()
model = model.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.)

In [6]:
def score(model, dataloader):
    # initital values
    calc_loss = 0.
    calc_correct = 0.    
    for data in dataloader: 
        # next batch
        inputs, labels = data
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())
        # model scores
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels.squeeze())
        # sum the loss and accuracy
        calc_loss += loss.data[0]
        calc_correct += torch.sum(preds == labels.squeeze().data) 
    count = len(dataloader.dataset)
    return round(calc_loss/count, 4), round(calc_correct/count, 4)

In [7]:
def train(trainloader, evalloader, validloader, epochs):
    for epoch in range(epochs):
        # initial values
        calc_loss = 0.
        calc_correct = 0.
        # read data
        for data in trainloader: 
            inputs, labels = data
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            # calculate loss and output
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels.squeeze())
            # clear gradients
            optimizer.zero_grad()
            # update weights
            loss.backward()
            optimizer.step()
            # track losses
            calc_loss += loss.data[0]
            calc_correct += torch.sum(preds == labels.squeeze().data)
        # evaluate
        count = len(trainloader.dataset)
        print('----------------')
        print('epoch:', epoch)
        print('run loss:', (round(calc_loss/count, 4), round(calc_correct/count, 4)))
        print('train loss:', score(model, evalloader))
        print('valid loss:', score(model, validloader))
        print('----------------')     
    return model

In [8]:
def adjust_lr(optimizer, value):
    for param_group in optimizer.param_groups:
        param_group['lr'] = value
    return optimizer

### execution

In [12]:
model

resnet (
  (dense1): Linear (2048 -> 500)
  (dropout): Dropout (p = 0.33)
  (batchnorm): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True)
  (dense2): Linear (500 -> 12)
)

In [13]:
optimizer = adjust_lr(optimizer, 0.0001)
model = train(train_resnet, train_resnet, valid_resnet, 1)

----------------
epoch: 0
run loss: (0.2033, 0.139)
train loss: (0.1992, 0.1681)
valid loss: (0.0385, 0.1217)
----------------


In [ ]:
optimizer = adjust_lr(optimizer, 0.0005)
model = train(train_resnet, train_resnet, valid_resnet, 1)

In [ ]:
optimizer = adjust_lr(optimizer, 0.0001)
model = train(train_resnet, train_resnet, valid_resnet, 2)

In [ ]:
optimizer = adjust_lr(optimizer, 0.00005)
model = train(train_resnet, train_resnet, valid_resnet, 2)

In [ ]:
optimizer = adjust_lr(optimizer, 0.00001)
model = train(train_resnet, train_resnet, valid_resnet, 2)